# Machine Learning

In [ ]:
%%time 

from IPython.display import display_html, clear_output, Markdown;
from gc import collect;
from copy import deepcopy;
import pandas as pd
from typing import Dict, List, Tuple;
import numpy as np;
from datetime import datetime;
import numpy as np;
import polars as pl
# import cudf
from pprint import pprint;
import os;
from functools import partial;
from collections import Counter;
from itertools import product;
from colorama import Fore, Style, init;
from warnings import filterwarnings;
filterwarnings('ignore');

from tqdm.notebook import tqdm;
import seaborn as sns;

import matplotlib.pyplot as plt;
%matplotlib inline

clear_output();
print();

In [ ]:
%%time 

# Data engineering and pipeline development:-
from sklearn.model_selection import (RepeatedStratifiedKFold as RSKF, 
                                     StratifiedKFold as SKF,
                                     KFold, 
                                     RepeatedKFold as RKF, 
                                     cross_val_score,
                                     train_test_split
                                    );

from sklearn.pipeline import Pipeline, make_pipeline;
from sklearn.base import BaseEstimator, TransformerMixin;
from sklearn.compose import ColumnTransformer;
from sklearn.metrics import average_precision_score as aps, accuracy_score, classification_report, mean_squared_error

# Model development:-
from xgboost import XGBClassifier;
from lightgbm import LGBMClassifier, log_evaluation, early_stopping;
from catboost import CatBoostClassifier;
from sklearn.ensemble import (RandomForestClassifier as RFC,
                              ExtraTreesClassifier as ETC,
                              GradientBoostingClassifier as GBC,
                              HistGradientBoostingClassifier as HGBC,
                             );
import pickle

In [ ]:
%%time
series = pd.read_parquet("/kaggle/input/detect-sleep-states-memory-decrease/train_series.parquet")
events = pd.read_parquet(r"/kaggle/input/detect-sleep-states-memory-decrease/train_events.parquet")
childsr = series[series['id_map'] == 127]
childev = events[events['id_map'] == 127]    


# series.drop(['timestamp'], axis=1, inplace=True)
series.drop('step', inplace=True, axis=1)
events.drop(['timestamp', 'step'], inplace=True, axis=1)
series.dropna(inplace=True)
# events.drop(timestamp, axis=1, inplace=True)
events.dropna(inplace=True)
series.reset_index(inplace=True, drop=True)
events.reset_index(inplace=True, drop=True)

holdout_frac = 0.2  # 20% of data as holdout set
series_train, series_holdout = train_test_split(series, test_size=holdout_frac, random_state=42)
events_train, events_holdout = train_test_split(events, test_size=holdout_frac, random_state=42)

In [ ]:
train_sr = pd.read_par

## Considering only 3 days for each child

In [ ]:
series.timestamp.dt.day.value_counts().plot(kind='bar')cha

In [ ]:
events.isna().sum()/len(events)

In [ ]:
events[events.isna()]

In [ ]:
# events.id_map.nunique()

In [ ]:
print("Series columns:", series.columns)
print("Events columns:", events.columns)

In [ ]:
# len(series[series.id_map == 3 ])

In [ ]:
# events[events.id_map == 3].step.max()

In [ ]:
events.head()

In [ ]:
series.head()

In [ ]:
childsr = series[series['id_map'] == 127]
childev = events[events['id_map'] == chaA]    

joined_frame = pd.merge(childsr, childev, on='id_map')
# joined_frame = pd.merge(events, series, on='id_map')
# joined_frame.drop(["id_map", "night"], axis=1, inplace=True)
# joined_frame.reset_index(inplace=True, drop=True)
print(joined_frame.shape)
joined_frame.sample(6)

In [ ]:
# child_ids = series.id_map.unique()
# for sample_percent in range(10, 100, 10):
#     sample_data = pd.DataFrame(columns = ['step_x', 'anglez', 'enmo', 'id_map', 'night', 'event', 'step_y'])
#     for child_id in tqdm(child_ids):
#         child_series = series_train[series.id_map == child_id].sample(frac=0.05)
#         child_events = events_train[events.id_map == child_id].sample(frac = 0.05)
        
#         joined_frame = pd.merge(childsr, childev, how='left', on='id_map')
    
#         sample_data = pd.concat([sample_data, joined_frame])
#     break

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# joined_frame = pd.merge(childsr, childev, how='left', on='id_map')
joined_frame.dropna(inplace=True)
joined_frame.reset_index(inplace=True, drop=True)

float16_columns = ['step', 'anglez', 'enmo', 'event']
joined_frame[float16_columns] = joined_frame[float16_columns].astype('float32')
print(joined_frame.shape)
joined_frame.head()


In [ ]:
X = joined_frame.drop("event", axis=1).to_numpy()
y = joined_frame.event.to_numpy()
X_train, X_val, y_train, y_val = train_test_split( X, y, test_size=0.2,random_state=42)

imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_val = imputer.fit_transform(X_val)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
# Check data balancing
# df = pd.DataFrame(list(y_train.items()), columns=['ID', 'Value'])

# plt.figure(figsize=(10, 6))
# sns.countplot(x='Value', data=df, palette='viridis')
# plt.title('Value Counts')
# plt.xlabel('Value')
# plt.ylabel('Count')
# plt.show()

### Base model (Gradient Boosting Classifier)

In [ ]:
n_splits = 30
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

gbc = GBC(n_estimators=500)
# gbc.fit(X_train, y_train)

# train_preds = gbc.predict(X_train)
# test_preds = gbc.predict(X_val)
# print("Train Classification report: ")
# print(classification_report(y_train, train_preds))
# print("Test Classification report: ")
# print(classification_report(y_val, test_preds))
mse_scores = []
acc_scores = []
best_acc = 0

for fold_num, (train_index, test_index) in tqdm(enumerate(kf.split(X), 1), total=n_splits):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    gbc.fit(X_train, y_train)
    y_pred = gbc.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    mse_scores.append(mse)
    acc_scores.append(acc)
    if best_acc < acc:
        best_acc = acc
        pickle.dump(gbc, open("GBC.pkl", 'wb'))
    print("Accuracy:", acc)
    print("MSE:", mse)
#     plt.figure(figsize=(10, 6))
#     plt.scatter(y_test, y_pred)
#     plt.title(f'Fold {fold_num} - True vs Predicted Values')
#     plt.xlabel('True Values')
#     plt.ylabel('Predicted Values')
#     plt.show()


n_splits = len(mse_scores)

fig, ax1 = plt.subplots(figsize=(10, 6))
ax1.plot(range(1, n_splits + 1), mse_scores, marker='o', linestyle='-', color='b', label='MSE')
ax1.set_xlabel('Fold Number')
ax1.set_ylabel('Mean Squared Error', color='b')
ax1.tick_params('y', colors='b')
ax1.set_title('Mean Squared Error and Accuracy for Each Fold')

ax2 = ax1.twinx()
ax2.plot(range(1, n_splits + 1), acc_scores, marker='o', linestyle='-', color='r', label='Accuracy')
ax2.set_ylabel('Accuracy', color='r')
ax2.tick_params('y', colors='r')

# Displaying the legend
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')

plt.show()


In [ ]:
# from tqdm import tqdm
# from sklearn.model_selection import KFold
# from sklearn.metrics import accuracy_score
# from sklearn.ensemble import StackingClassifier
# from sklearn.linear_model import LogisticRegression

# # Define base classifiers
# base_classifiers = [
#     ('xgb', XGBClassifier()),
#     ('lgbm', LGBMClassifier()),
#     ('catboost', CatBoostClassifier(verbose=False)),
#     ('rfc', RFC()),
#     ('etc', ETC()),
#     ('gbc', GBC()),
#     ('hgbc', HGBC())
# ]

# # Create a stacking classifier with a logistic regression meta-classifier
# stacking_classifier = StackingClassifier(estimators=base_classifiers,
#                                          final_estimator=LogisticRegression(),
#                                          cv=KFold(n_splits=25, shuffle=True, random_state=42))

# # Perform KFold cross-validation
# kf = KFold(n_splits=25, shuffle=True, random_state=42)

# accuracies = []
# errors = []
# for train_index, test_index in tqdm(kf.split(X), total=kf.get_n_splits(), desc='KFold Progress'):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

#     # Fit the stacking classifier on the training set
#     stacking_classifier.fit(X_train, y_train)

#     # Make predictions on the test set
#     y_pred = stacking_classifier.predict(X_test)

#     # Evaluate the accuracy
#     accuracy = accuracy_score(y_test, y_pred)
#     error = mean_squared_error(y_test, y_pred)
    
#     accuracies.append(accuracy)
#     errors.append(error)
#     print(f'Accuracy: {accuracy}')


In [ ]:
# %%time
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.linear_model import LogisticRegressionCV
# from sklearn.naive_bayes import MultinomialNB

# rf_model = RFC(n_estimators=10, max_depth=10)
# knn_model = KNeighborsClassifier(n_neighbors=5)
# lr_model = LogisticRegressionCV(cv=4)
# mnb = MultinomialNB()

# rf_model.fit(X_train, y_train)
# lr_model.fit(X_train, y_train)
# knn_model.fit(X_train, y_train)

# y_pred_rf = rf_model.predict(X_val)
# y_pred_knn = knn_model.predict(X_val)

# X_stack = np.column_stack((y_pred_rf, y_pred_knn))

# final_model = RFC(n_estimators=500, max_depth=10)
# final_model.fit(X_stack, y_val)

# y_pred_stacked = final_model.predict(X_stack)

# accuracy = accuracy_score(y_val, y_pred_stacked)
# print(f'Stacked Model Accuracy: {accuracy:.4f}')

In [ ]:
# from sklearn.metrics import roc_curve, auc, classification_report
# import matplotlib.pyplot as plt

# y_prob = final_model.predict_proba(X_val)[:, 1]

# fpr, tpr, thresholds = roc_curve(y_val, y_prob)
# roc_auc = auc(fpr, tpr)

# plt.figure(figsize=(8, 6))
# plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = {:.2f})'.format(roc_auc))
# plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic (ROC) Curve')
# plt.legend(loc='lower right')
# plt.show()

# threshold = 0.5
# y_pred_binary = (y_prob > threshold).astype(int)

# class_report = classification_report(y_val, y_pred_binary)
# print(f'Classification Report:\n{class_report}')


# Plot decision boundaries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from mlxtend.data import iris_data
from mlxtend.plotting import plot_decision_regions

# Initializing Classifiers
clf1 = LogisticRegression(random_state=0)
clf2 = RandomForestClassifier(random_state=0)
clf3 = SVC(random_state=0, probability=True)
eclf = EnsembleVoteClassifier(clfs=[clf1, clf2, clf3], weights=[2, 1, 1], voting='soft')

# Loading some example data
X, y = iris_data()
X = X[:,[0, 2]]

# Plotting Decision Regions
gs = gridspec.GridSpec(2, 2)
fig = plt.figure(figsize=(10, 8))

for clf, lab, grd in zip([clf1, clf2, clf3, eclf],
                         ['Logistic Regression', 'Random Forest', 'RBF kernel SVM', 'Ensemble'],
                         itertools.product([0, 1], repeat=2)):
    clf.fit(X, y)
    ax = plt.subplot(gs[grd[0], grd[1]])
    fig = plot_decision_regions(X=X, y=y, clf=clf, legend=2)
    plt.title(lab)
plt.show()

# Creating test submission

In [ ]:
event_mapper = lambda ev: 'onset' if ev==1 else "wakeup"

In [ ]:
test_series = pd.read_parquet("/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet")
sr_id = test_series.series_id
test_sub = pd.DataFrame({"series_id": sr_id})
test_series.drop(['series_id', "timestamp"], axis=1, inplace=True)
test_series.sample(4)

In [ ]:
test_series.reset_index(inplace=True, drop=True)
model = pickle.load(open("GBC.pkl", 'rb'))
test_sub['event'] = gbc.predict(test_series)
test_sub['score'] = gbc.predict_proba(test_series)[:, 1]
test_sub['event'] = test_sub['event'].apply(event_mapper)
test_sub['step'] = test_series['step']
test_sub['row_index'] = test_sub.index
test_sub.sample(4)

In [ ]:
test_sub.to_csv('submission.csv', index=False)
pd.read_csv('submission.csv')

In [ ]:
pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv")